# Title
[]()

In [6]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
# sys.path.append(r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\GHL-chat\src\app')
from silvhua import *

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [8]:
from tabula import read_pdf
import json
import re
import warnings
def parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all', 
    # last_id_column='Major Heading', value_to_ignore=None,
    # blank_columns=None,
    ):
    """
    Create dataframe from a PDF file using 
    """
    print(f'Extracting the following files:')
    if type(pdf_filenames) != list:
        pdf_filenames = [pdf_filenames]
    dfs_list = []
    reference_dict = dict()
    corrected_pages = dict()
    for index, file in enumerate(pdf_filenames):
        print(file)
        filepath = f'{filepath}/'.replace('\\','/') if filepath else ''
        path_and_file = filepath+file
        try:
            df_list = read_pdf(path_and_file, pages=pages)
            dfs_list.append(df_list)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            message = f'An error occurred on line {lineno} in {filename}: {error}.'
            print(message)
    return dfs_list
        # file_dfs = []
        # page1_columns = df_list[0].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True).to_list()
        # print('number of columns:', len(page1_columns))
        # if last_id_column in page1_columns:
        #     page1_columns_sorted = sorted(page1_columns)
        # else:
        #     previous_columns = dfs_list[index-1].columns
        #     print(f'\t\tNo headers found; using headers from previous document:', previous_columns)
        #     if len(df_list[page_number].columns) > len(previous_columns):
        #         n_cols_to_pad = len(df_list[page_number].columns) - len(previous_columns)
        #         new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
        #         page1_columns_sorted = sorted(page1_columns) + new_columns_padding
        #         page1_columns = page1_columns + new_columns_padding

        #     elif len(df_list[page_number].columns) < len(previous_columns):
        #         page1_columns = (previous_columns[:len(df_list[page_number].columns)].to_list())
        #         page1_columns_sorted = sorted(page1_columns)
        #     else:
        #         page1_columns_sorted = (dfs_list[index-1].columns.to_list())
        #     page1_columns = ['Unnamed: 0' if column=='index' else column for column in page1_columns]
        #     print(f'\t\t**Current df columns:',page1_columns)
        
        # for page_number in range(len(df_list)):
        #     print(f'\tpage {page_number+1}...')
        #     df_list[page_number].columns = df_list[page_number].columns.str.replace('\r', '').str.replace(r'(workplace)([a-zA-Z]+)', r'\1_\2', regex=True)
        #     print('\t\tCurrent number of columns:', len(df_list[page_number].columns), len(page1_columns_sorted))
        #     print(f'\t\t{len(page1_columns_sorted)}, {len(page1_columns)}')
        #     if len(df_list[page_number].columns.to_list()) == len(page1_columns_sorted):
        #         if sorted(df_list[page_number].columns.to_list()) == page1_columns_sorted:
        #             pass
        #         else:
        #             df_list[page_number].columns = page1_columns 
        #             print('\t\tKeeping page 1 columns')
        #     else:
        #         if len(df_list[page_number].columns) > len(page1_columns_sorted):
        #             n_cols_to_pad = len(df_list[page_number].columns) - len(page1_columns_sorted)
        #             new_columns_padding = [f'blank{index}' for index, col in enumerate(range(n_cols_to_pad))]
        #             df_list[page_number].columns = page1_columns + new_columns_padding
        #             print('\t\tPadding columns')
        #         elif len(df_list[page_number].columns) < len(page1_columns_sorted):
        #             df_list[page_number].columns = page1_columns[:len(df_list[page_number].columns)]
        #             print('\t\tDropping columns')
        #         else:
        #             df_list[page_number].columns = page1_columns
        #         corrected_pages[file] = page_number+1
        #         print(f'\n\t**Fixing column parsing for page {page_number+1} of {file}**')
        #         print(f'\t\t{df_list[page_number].columns}')
        #     # Consolidate rows that were incorrectly split
        #     if (df_list[page_number]['Unnamed: 0'].isna().sum() > 0):
        #         # print('\tAttempting to consolidate rows that were incorrectly split: ', df_list[page_number].columns)
        #         print('\tAttempting to consolidate rows that were incorrectly split: ')
        #         try:
        #             df_list[page_number] = merge_rows(df_list[page_number], file)
        #         except:
        #             print(f'\t\tUnable to consolidate rows on page {page_number+1} of {file}')
        #             reference_dict['unparsed documents'] = file
        #             continue

        #     # Align the columns
        #     if value_to_ignore in df_list[page_number].iloc[0].values:
        #         value_to_ignore_column = df_list[page_number].iloc[0].values.tolist().index(value_to_ignore)
        #         print(f'\tRemoving {value_to_ignore} from column {value_to_ignore_column}')
        #         df_list[page_number].iloc[0, value_to_ignore_column:-1] = df_list[page_number].iloc[0, value_to_ignore_column+1:].values

        #     for column in blank_columns:
        #         try:
        #             if df_list[page_number][column].isna().sum() != len(df_list[page_number]): # 2023-03-30 22:45:
        #                 blank_column_index = df_list[page_number].columns.tolist().index(column)
        #                 print(f'\tShifting data from column {blank_column_index}({column})')
        #                 df_list[page_number].iloc[:,blank_column_index+1:] = df_list[page_number].iloc[:,blank_column_index:-1].values 
        #                 df_list[page_number].iloc[:,blank_column_index] = None
        #         except:
        #             pass

        #     file_dfs.append(df_list[page_number]) # 2023-03-30 23:04 Decrease indent
        # dfs_list.append(file_dfs) # SH 2023-04-01 20:58

pdf_filenames = ['1_2024-adult-compendium_1_2024.pdf']

dfs_list = parse_pdf_tolist(
    pdf_filenames, 
    filepath=r'C:\Users\silvh\OneDrive\Ginkgo\references', 
    pages='all')
dfs_list

Extracting the following files:
1_2024-adult-compendium_1_2024.pdf
An error occurred on line 26 in C:\Users\silvh\AppData\Local\Temp\ipykernel_6156\2345812797.py: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`.


[]

# *End of Page*